In [ ]:
from IPython.display import HTML, display

# <font color='green'>Integrate and Fire Neuron Model </font>

Hodgkin-Huxley model is a complete mathematical in the sense that it provides mathematical equations that fit well with the voltage and time dependence of two most promiment currents involved in the generation of the action potential. But this is a highly complex and non-linear model to allow for mathematical analysis both at single neuron level and at the network level. Moreover, computationally, HH model is very expensive, especially because the ODEs describing the dynamical can be stiff.

Therefore, simple neuron models have been proposed. Linear integrate and fire model is one the simplest neuron model. It assumed that the Na and K conductances that constitute major non-linearities in the HH model are essentially only for setting up the spike threshold and spike shape. In most conditions, spike threshold is largely fixed and spike shape does not seem to tbe very important for information processing. Therefore, the Na and K conductance can be replaced by a fixed threshold $V_{th}$.

Because, during the spike a neuron does not integrate synaptic inputs, the IAF neurons membrane potential is clamped to the resting membrane potential (the so-called refractory period). As soon as the neuron hits the threshold, a spike is registerd. 

## <font color='green'>Aims of the tutorial </font>
### Get Familarized with a biological neuronal network simulation tool NEST
### Simulate a Integrate-and-fire neuron model
### Generate the input current and output firing rate curve of the IAF neuron model
### Compare a single IAF and a population of IAFs in terms of how fast stimulus they can encode

## <font color='green'>Learnign outcomes</font>
### You should be able to write a simple code in NEST
### Understand the limitations and strengths of the IAF neuron model

## <font color='green'>What will you do</font>
### Execute the Python code and display the results and describe your observations
### Write small code snippetes to perform certain simulations in loop

## <font color='green'>Equivalent electrical circuit</font>
A neuron membrane can be described as an electrical circuit with batteris, voltage dependent resistors and a capacitor. 
<img src="./Figures/iaf_neuron_synapse_circuit_model.png">
The full figure about reprsents the HH model with synaptic inputs modelled as current or conductances. For the Integrate-and-fire neuron we do not consider the voltage dependent Na and K conductance (those covered with the gray box)
-  $C_m$ is the passive membrane capacitance
-  $g_K$ and $g_{NA}$ the voltage-gated ion channels have been removed
-  $g_L$ is the passive leak channel which accounrs for baseline flow of ions across the membrane and is represented by a linear conductance
-  $E_L$ Leak voltage or resting membrane potential
-  Synapses can be connected as we did for the HH model
- $g_{exc}$ is the excitatory synaptic conductance -- the so called 'conductance-based' synapse model
- $g_{inh}$ is the inhibitory synaptic conductance -- the so called 'conductance-based' synapse model
- $I_{exc}$ and $I_{inh}$ inhibotory synaptic currents -- when synapses are modelled as 'current-injections' as opposed to 'conductance changes'


## <font color='green'>Mathematical equation</font>

The differential equations governing the dynamics of voltage is given by:

when $V < V_{th}$ and $is\_ref=0$
\begin{eqnarray*}
C_m\frac{dV}{dt} & = & g_L(V-E_L) + I_{inj} + I_{syn}\\
\end{eqnarray*}
when $V \geq V_{th}$ and $is\_ref < t_{ref}$ $V \rightarrow E_L$

when $V \geq V_{th}$ and $is\_ref = t_{ref}$ $V\rightarrow E_L$ and $t_{spike} = 1$

-  $I_{inj}$ is the injected current
- $I_{syn}$ is total synaptic current (whether generates by exc or inh synapses)
-  $\bar{g_L}$ are the maximum conductances for a given channel (sodium Na, potassium K, and leak L)
-  $E_L$ are the leak voltage or the resting membrane potential 

In [ ]:
import numpy as np
import pylab as plt
import matplotlib as matplt
%matplotlib inline
import nest
import time
np.random.seed(1000)
nest.ResetKernel()

## <font color='green'>Create a neuron in NEST</font>

In [ ]:
# Create a neuron in NEST
# Lets create a single IAF neuron to which you can conect current-based synapses (More on this later)
iaf_neuron = nest.Create('iaf_psc_alpha',1) 

# To know the parameters of the neuron
nest.GetStatus(iaf_neuron)

## <font color='green'>Changing neuron parameters in NEST</font>

In [ ]:
# In case you want to change the To change the neuron parameters
neuron_params = {'V_th': -54., 
                 'V_reset': -70.0, 
                 't_ref': 2.0, 
                 'E_L' : -70.,
                 'C_m': 200.}
nest.SetStatus(iaf_neuron,neuron_params)
nest.GetStatus(iaf_neuron,{'C_m'})

## <font color='green'>Input current and output firing rate response of an IAF neuron</font>

In [ ]:
# Create a current generator to inject current

dc_params = {'amplitude': 340.,
            'start': 100.,
            'stop': 600.}

dc_gen = nest.Create('dc_generator',1)
nest.SetStatus(dc_gen,dc_params)
nest.GetStatus(dc_gen)

# Connect the dc-genertor to the neuron
nest.Connect(dc_gen,iaf_neuron)

# Also you would like to record the neuron membrane potential and spikes
# So we connect a voltmeter 

multimeter = nest.Create("multimeter")
nest.SetStatus(multimeter, {"withtime":True, "record_from":["V_m"]})

# and spike detector to the neuron
spikedetector = nest.Create("spike_detector",
                params={"withgid": True, "withtime": True})

# Finally connect them to the neuron
nest.Connect(multimeter, iaf_neuron)
nest.Connect(iaf_neuron, spikedetector)

In [ ]:
# Simulate the neuron
nest.Simulate(1000.) # Simulation time in msec


In [ ]:
# And display the membrane potential
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]

# And get the spike events
dSD = nest.GetStatus(spikedetector,keys="events")[0]
evs = dSD["senders"]
spk_ts = dSD["times"]

plt.figure(figsize=(20, 5))
plt.plot(mem_ts,Vms,linewidth=2,label='voltage')
plt.plot(spk_ts, np.ones(len(spk_ts))*-54., ".",ms=20)
plt.ylim(-75,-50)
plt.xlim(0,1000)
plt.legend()
plt.ylabel('Voltage [mV]')
plt.xlabel('Time [msec]')
plt.legend()
plt.show()
#plt.show()

# <font color=blue>Describe</font>
### From the above what are the similarities and differences between IAF and HH models
### OR what features of a biological neuron you think are being captured by the IAF neuron


# <font color=salmon>TO DO</font>
## Generate the input current and output firing rate curve of the IAF neuron 
#### <font color=gray> Write a code that increases the injected current amplitude and counts the number of spikes. </font>
#### Note that everytime you call the NEST function nest.simulate the internal clock is advanced. So you need to keep track of the time at which you simulator is operating

In [ ]:
nest.SetStatus(dc_gen,{'stop':1e8}) # the stimulator is on forever
stim_duration = 200. # msec
inter_stim_period = 500. # to cool down the neuron so it goes back to resting membrane potential

stim_amplitude = np.linspace(320.,600.,50)
dmm = nest.GetStatus(multimeter)[0]
mem_ts = dmm["events"]["times"]
last_time_stamp = mem_ts[-1]

spk_count = np.zeros(len(stim_amplitude))
for ii in range(len(stim_amplitude)):
    nest.SetStatus(dc_gen,{'amplitude':stim_amplitude[ii]})
    nest.Simulate(stim_duration)
    # extract spikes that occured in the previous stim_duration
    # Get the mempot
    dmm = nest.GetStatus(multimeter)[0]
    mem_ts = dmm["events"]["times"]
    last_time_stamp = mem_ts[-1]
    # get the spikes
    dSD = nest.GetStatus(spikedetector,keys="events")[0]
    spk_ts = dSD["times"]
    spk_in_previous_stim = spk_ts[spk_ts>=last_time_stamp - stim_duration]
    spk_count[ii] = len(spk_in_previous_stim)
    nest.SetStatus(dc_gen,{'amplitude':0.}) # reduce the 
    nest.Simulate(inter_stim_period)

In [ ]:
# Get the mempot
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]

plt.figure(figsize=(20, 5))

plt.subplot(2,1,1)
plt.plot(mem_ts,Vms,linewidth=2,label='voltage')
plt.ylim(-75,-50)
plt.subplot(2,1,2)
plt.plot(stim_amplitude,spk_count)

# IAF Neuron with noise inputs

In [ ]:
nest.ResetKernel()
iaf_neuron = nest.Create('iaf_psc_alpha',1) 
# In case you want to change the To change the neuron parameters
neuron_params = {'V_th': -54., 
                 'V_reset': -70.0, 
                 't_ref': 2.0, 
                 'E_L' : -70.,
                 'C_m': 200.}
nest.SetStatus(iaf_neuron,neuron_params)
nest.GetStatus(iaf_neuron,{'C_m'})
# Create a current generator to inject current

# initially we set it to zero so we can change in the loop
noise_params = {'mean': 300.,
                'std': 100.
               }

noise_gen = nest.Create('noise_generator',1)
nest.SetStatus(noise_gen,noise_params)
nest.GetStatus(noise_gen)

# Connect the dc-genertor to the neuron
nest.Connect(noise_gen,iaf_neuron)

# Also you would like to record the neuron membrane potential and spikes
# So we connect a voltmeter 

multimeter = nest.Create("multimeter")
nest.SetStatus(multimeter, {"withtime":True, "record_from":["V_m"]})

# and spike detector to the neuron
spikedetector = nest.Create("spike_detector",
                params={"withgid": True, "withtime": True})

# Finally connect them to the neuron
nest.Connect(multimeter, iaf_neuron)
nest.Connect(iaf_neuron, spikedetector)

nest.Simulate(1000.)

In [ ]:
# And display the membrane potential
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]

# And get the spike events
dSD = nest.GetStatus(spikedetector,keys="events")[0]
evs = dSD["senders"]
spk_ts = dSD["times"]

plt.figure(figsize=(20, 5))
plt.plot(mem_ts,Vms,linewidth=2,label='voltage')
plt.plot(spk_ts, np.ones(len(spk_ts))*-54., ".",ms=20,label='spikes')
plt.ylim(-75,-50)
plt.xlim(0,1000)
plt.ylabel('Voltage [mV]')
plt.xlabel('Time [msec]')
plt.legend()
plt.show()
#plt.show()

# Transfer function for noise injection
## We will reset the nest kernel and recreate the devices etc. otherwise the nest clock will go very far in time
### Change the mean and standard deviation of the input and count the spikes
### In the following we will only change the mean

In [ ]:

stim_duration = 200. # msec
inter_stim_period = 500. # to cool down the neuron so it goes back to resting membrane potential

stim_mean = np.linspace(320.,600.,10)
dmm = nest.GetStatus(multimeter)[0]
mem_ts = dmm["events"]["times"]
last_time_stamp = mem_ts[-1]

spk_count = np.zeros(len(stim_mean))
for ii in range(len(stim_mean)):
    nest.SetStatus(noise_gen,{'mean':stim_mean[ii],'std':100.})
    nest.Simulate(stim_duration)
    # extract spikes that occured in the previous stim_duration
    # Get the mempot
    dmm = nest.GetStatus(multimeter)[0]
    mem_ts = dmm["events"]["times"]
    last_time_stamp = mem_ts[-1]
    # get the spikes
    dSD = nest.GetStatus(spikedetector,keys="events")[0]
    spk_ts = dSD["times"]
    spk_in_previous_stim = spk_ts[spk_ts>=last_time_stamp - stim_duration]
    spk_count[ii] = len(spk_in_previous_stim)
    
    nest.SetStatus(noise_gen,{'mean':0.,'std':0.})
    nest.Simulate(inter_stim_period)
    

In [ ]:
print(len(stim_mean))
print(len(spk_count))

In [ ]:
# Get the mempot
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
mem_ts = dmm["events"]["times"]

plt.figure(figsize=(20, 5))

plt.subplot(2,1,1)
plt.plot(mem_ts,Vms,linewidth=2,label='voltage')
plt.ylim(-75,-50)
plt.subplot(2,1,2)
plt.plot(stim_mean,spk_count)

# <font color=blue>Describe</font>
### What is the main difference between DC injection and Noise Injection in terms of the IAF transfer function shapes

# Stimulus Encoding by a population of neurons 
### Now we will test how fast a population neurons driven by either DC input or noisy input can respond to a stimulus pulse.

In [ ]:
nest.ResetKernel()
# Lets create 100 neurons
N = 100
iaf_neuron = nest.Create('iaf_psc_alpha',N) 
# Each neuron is identical 
neuron_params = {'V_th': -54., 
                 'V_reset': -70.0, 
                 't_ref': 2.0, 
                 'E_L' : -70.,
                 'C_m': 200.}
nest.SetStatus(iaf_neuron,neuron_params)

# In this setup we will firt inject DC in all the neurons and then Noise
# In the presence of each we will inject a small amount of dc pulse -- lets call it dc_signal
# The DC
dc_params = {'amplitude': 340.,
            'start': 100.,
            'stop': 900.}
dc_gen = nest.Create('dc_generator',1)
nest.SetStatus(dc_gen,dc_params)

# The noise
noise_params = {'mean': 300.,
                'std': 100.,
                'start': 1000.,
                'stop': 1800.,
               }
noise_gen = nest.Create('noise_generator',1)
nest.SetStatus(noise_gen,noise_params)

signal_amp = 50.
dc_stim_start = 270.
dc_stim_end = 370.
noise_stim_start = 1270.
noise_stim_end = 1370.

# The signal -- this will injected from 200 to 700 ms and then from 1100 to 1600ms
signal_params_1 = {'amplitude': signal_amp,
            'start': dc_stim_start,
            'stop': dc_stim_end}
signal_gen_1 = nest.Create('dc_generator',1)
nest.SetStatus(signal_gen_1,signal_params_1)

# THis is identical to the above one but will switch on at a different time
signal_params_2 = {'amplitude': signal_amp,
            'start': noise_stim_start,
            'stop': noise_stim_end}
signal_gen_2 = nest.Create('dc_generator',1)
nest.SetStatus(signal_gen_2,signal_params_2)

# Connect the dc-genertor to the neuron
nest.Connect(noise_gen,iaf_neuron)
nest.Connect(dc_gen,iaf_neuron)
nest.Connect(signal_gen_1,iaf_neuron)
nest.Connect(signal_gen_2,iaf_neuron)

# Now we need to record the spike detector to the neuron
spikedetector = nest.Create("spike_detector",
                params={"withgid": True, "withtime": True})

# Finally connect them to the neuron
nest.Connect(iaf_neuron, spikedetector)

nest.Simulate(1500.)


In [ ]:
dSD = nest.GetStatus(spikedetector,keys="events")[0]
evs = dSD["senders"]
spk_ts = dSD["times"]
bin_width = 2.#msec
ed = np.arange(0.,2000.,bin_width)
hh,ed1 = np.histogram(spk_ts,ed)
hh = 1e3*hh/np.float(N)/bin_width

dc_tx = np.arange(0,1000,1)
dc_stim = np.zeros(np.size(dc_tx))
dc_stim[int(dc_stim_start):int(dc_stim_end)]=20

noise_tx = np.arange(1000,2000,1)
noise_stim = np.zeros(np.size(dc_tx))
noise_stim[int(noise_stim_start)-1000:int(noise_stim_end)-1000]=20


plt.figure(figsize=(20, 10))
plt.subplot(2,2,1)
plt.plot(spk_ts,evs,'.')
plt.xlim(0,1000)
plt.xlabel('Time [msec]')
plt.ylabel('Neuron id.')
plt.xlim(200,400)

plt.subplot(2,2,3)
plt.plot(ed1[:-1],hh)
plt.plot(dc_tx,dc_stim,lw=2)
plt.xlim(0,1000)
plt.ylabel('Spike rate')
plt.xlim(200,400)

#plt.legend('Average response firing rate','Input')

plt.subplot(2,2,2)
plt.plot(spk_ts,evs,'.')
plt.xlim(1000,2000)
plt.xlabel('Time [msec]')
plt.ylabel('Neuron id.')
plt.xlim(1200,1400)

plt.subplot(2,2,4)
plt.plot(ed1[:-1],hh)
plt.xlim(1000,2000)
plt.ylim(0,100)
plt.plot(noise_tx,noise_stim,lw=2)
plt.ylabel('Spike rate')
#plt.legend('Average response firing rate','Input')
plt.xlim(1200,1400)


# <font color=blue>Describe</font>
### What is the difference in the response of a population of IAF neurons deriven by DC input or by Noise input

# <font color=salmon>TO DO</font>
## In the above explore the role of changing the mean and variance of the noise input on the speed of response. The expectation is that as you increase the std of the noise the population become faster

# <font color=salmon>TO DO</font>
## Summarize your observations and conclusions.